In [ ]:
import os
import numpy as np
import open3d
import time
from utils.metric import ConfusionMatrix
import multiprocessing

In [ ]:
def load_labels(label_path):
    labels = []
    with open(label_path, "r") as f:
        for line in f:
            labels.append(int(float(line.strip())))
    return labels

def load_points(points_path):
    """
    v, x, y, z, r, g, b
    """
    points = []
    with open(points_path, 'r') as f:
        for line in f:
            line = line.strip()
            tokens = line.split()
            x = float(tokens[1])
            y = float(tokens[1])
            z = float(tokens[1])
            points.append([x, y, z])
    return np.array(points)

In [ ]:
project_root_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/"
sparse_labels_dir = os.path.join(project_root_dir, "visu/semantic_test/full_scenes_predictions/")
dense_labels_dir = os.path.join(project_root_dir, "visu/semantic_test/full_scenes_predictions_all_points/")
dense_points_dir = "/home/ylao/data/semantic3d"
gt_dir = "/home/ylao/data/semantic3d" # both points and labels

In [ ]:
file_prefix = "untermaederbrunnen_station1_xyz_intensity_rgb"

sparse_labels_path = os.path.join(sparse_labels_dir, file_prefix + "_pred.txt")
sparse_labels = np.array(load_labels(sparse_labels_path))

sparse_points_path = os.path.join(sparse_labels_dir, file_prefix + "_aggregated.txt")
sparse_points = load_points(sparse_points_path)

dense_points_path = os.path.join(dense_points_dir, file_prefix + ".pcd")
pcd_dense = open3d.read_point_cloud(dense_points_path)

gt_labels_path = os.path.join(gt_dir, file_prefix + ".labels")
gt_labels = load_labels(gt_labels_path)

In [ ]:
# Build KNN tree
pcd_sparse = open3d.PointCloud()
pcd_sparse.points = open3d.Vector3dVector(sparse_points)
pcd_sparse_tree = open3d.KDTreeFlann(pcd_sparse)

In [ ]:
def match_knn_label(dense_index):
    global dense_points
    global sparse_labels
    global pcd_sparse_tree
    
    dense_point = dense_points[dense_index]
    k, sparse_indexes, _ = pcd_sparse_tree.search_knn_vector_3d(dense_point, 10)
    knn_sparse_labels = sparse_labels[sparse_indexes]
    dense_label = np.bincount(knn_sparse_labels).argmax()
    return dense_label

# Assign labels
start = time.time()
dense_points = np.asarray(pcd_dense.points)
dense_indexes = list(range(len(dense_points)))
with multiprocessing.Pool() as pool:
    dense_labels = pool.map(match_knn_label, dense_indexes)
print("knn match time: ", time.time() - start)

# Eval
cm = ConfusionMatrix(9)
cm.increment_from_list(gt_labels, dense_labels)
cm.print_metrics()

In [ ]:
a = np.array([[1, 2 , 3], [0, 5, 6], [7, 8, 9]])
any(a[:, 0] == 0)